In [1]:
from pytezos import ContractInterface, pytezos, MichelsonRuntimeError
from decimal import Decimal
import time
from settings import settings

run_time = int(time.time())
one_hour = 60*60

In [2]:
contract_fn = 'crystal_ball_turbo.tz'
participants = dict(
    a='tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ',
    b='tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos',
    c='tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE'
)
contract = ContractInterface.from_file(contract_fn)

In [3]:
storage = {
    'currencyPair': 'XTZ-USD',
    'createdTime': run_time,
    'targetDynamics': 1_000_000,
    'betsCloseTime': run_time + 24*one_hour,
    'measureStartTime': 0,
    'measureOracleStartTime': 0,
    'isMeasurementStarted': False,
    'startRate': 0,
    'measurePeriod': 12*one_hour,
    'isClosed': False,
    'closedTime': 0,
    'closedOracleTime': 0,
    'closedRate': 0,
    'closedDynamics': 0,
    'isBetsForWin': False,

    'betsForLedger': {},
    'betsAgainstLedger': {},
    'liquidityLedger': {},

    'oracleAddress': 'KT1SUP27JhX24Kvr11oUdWswk7FnCW78ZyUn',

    'betsForSum': 0,
    'betsAgainstSum': 0,
    'liquiditySum': 0,

    'liquidityPercent': 0,
    'measureStartFee': 200_000,
    'expirationFee': 100_000
}

In [4]:
# participant A: adding liquidity 50/50 just at start:
res = contract.bet(betAgainst=50_000, betFor=50_000).with_amount(100_000).interpret(
    storage=storage, sender=participants['a'], now=run_time)

In [5]:
assert res.storage['betsForSum'] == 50_000
assert res.storage['betsAgainstSum'] == 50_000
assert len(res.storage['betsForLedger']) == 1
assert len(res.storage['betsAgainstLedger']) == 1
assert len(res.storage['liquidityLedger']) == 1
assert res.storage['liquidityLedger'][participants['a']] == 50_000

In [6]:
# TODO: check assertRaises contract.bet(betAgainst=0, betFor=50_000).with_amount(100_000)

In [7]:
# participant B: bets for 10_000 after 1 hour
res = contract.bet(betAgainst=0, betFor=50_000).with_amount(50_000).interpret(
    storage=res.storage, sender=participants['b'], now=run_time + one_hour)

assert res.storage['betsForSum'] == 100_000
assert res.storage['betsAgainstSum'] == 50_000
assert len(res.storage['betsForLedger']) == 2
assert len(res.storage['betsAgainstLedger']) == 1
assert len(res.storage['liquidityLedger']) == 1

In [8]:
res.storage['liquidityLedger']

{'tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ': 50000}

In [9]:
# participant A: adding more liquidity after 12 hours (exactly half of the betting period):
res = contract.bet(betAgainst=100_000, betFor=50_000).with_amount(150_000).interpret(
    storage=res.storage, sender=participants['a'], now=run_time + 12*one_hour)

assert res.storage['betsForSum'] == 150_000
assert res.storage['betsAgainstSum'] == 150_000
assert len(res.storage['betsForLedger']) == 2
assert len(res.storage['betsAgainstLedger']) == 1
assert len(res.storage['liquidityLedger']) == 1
assert res.storage['liquiditySum'] == 50_000 + 25_000

In [10]:
res.storage['liquiditySum']

75000

In [11]:
# participant C: adding more liquidity at the very end:
res = contract.bet(betAgainst=500_000, betFor=500_000).with_amount(1_000_000).interpret(
    storage=res.storage, sender=participants['c'], now=run_time + 24*one_hour)

assert res.storage['betsForSum'] == 650_000
assert res.storage['betsAgainstSum'] == 650_000
assert len(res.storage['betsForLedger']) == 3
assert len(res.storage['betsAgainstLedger']) == 2
assert len(res.storage['liquidityLedger']) == 2
assert res.storage['liquiditySum'] == 50_000 + 25_000 + 0

In [12]:
assert res.storage['betsForSum'] == sum(res.storage['betsForLedger'].values())
assert res.storage['betsAgainstSum'] == sum(res.storage['betsAgainstLedger'].values())
assert res.storage['liquiditySum'] == sum(res.storage['liquidityLedger'].values())

In [13]:
# TODO: test trying to close before any measurement starts:
# TODO: assert raises MichelsonRuntimeError
_ = """
callback_values = {
    'currencyPair': storage['currencyPair'],
    'lastUpdate': run_time + 24*one_hour,
    'rate': 6_000_000
}

res = contract.closeCallback(callback_values).interpret(
    storage=res.storage, sender=storage['oracleAddress'], now=run_time + 24*one_hour)
"""

In [14]:
# TODO trying to call startMeasurement before time comes (before 24*one_hour), assertRaises

In [15]:
# running start measurement after 26 hours:
res = contract.startMeasurement().interpret(
    storage=res.storage, sender=participants['a'], now=run_time + 26*one_hour)

assert len(res.operations) == 1
operation = res.operations[0]
assert operation['destination'] == storage['oracleAddress']
assert operation['parameters']['entrypoint'] == 'get'
assert operation['parameters']['value']['args'][0]['string'] == storage['currencyPair']

In [16]:
assert not res.storage['isMeasurementStarted']

# emulating callback from oracle 26 hours late (but call last value
# in oracle is still from prev hour):
start_running_time = run_time + 26*one_hour
start_oracle_time = start_running_time - 1*one_hour

callback_values = {
    'currencyPair': storage['currencyPair'],
    'lastUpdate': start_oracle_time,
    'rate': 6_000_000
}

# TODO trying to call startMeasurementCallback from address different than oracle, assertRaises
res = contract.startMeasurementCallback(callback_values).interpret(
    storage=res.storage, sender=storage['oracleAddress'],
    now=start_running_time, source=participants['a'])
assert len(res.operations) == 1
assert res.storage['startRate'] == callback_values['rate']
assert res.storage['isMeasurementStarted']
assert res.storage['measureStartTime'] == start_running_time
assert res.storage['measureOracleStartTime'] == start_oracle_time

operation = res.operations[0]
assert operation['destination'] == participants['a']
assert int(operation['amount']) == storage['measureStartFee']

In [17]:
# calling close, should create opearaton with call to oracle get
res = contract.close().interpret(storage=res.storage, sender=participants['b'])
assert len(res.operations) == 1
operation = res.operations[0]
assert operation['destination'] == storage['oracleAddress']
assert operation['parameters']['entrypoint'] == 'get'
assert operation['parameters']['value']['args'][0]['string'] == storage['currencyPair']

In [18]:
# emulating callback from oracle 38 (24+12+2) hours late (but call last value
# in oracle is still from prev hour):

close_running_time = run_time + 38*one_hour
close_oracle_time = close_running_time - 1*one_hour

# price is increased 25%:
callback_values = {
    'currencyPair': storage['currencyPair'],
    'lastUpdate': close_oracle_time,
    'rate': 7_500_000
}

res = contract.closeCallback(callback_values).interpret(
    storage=res.storage, sender=storage['oracleAddress'],
    now=close_running_time, source=participants['b'])
assert len(res.operations) == 1
assert res.storage['closedRate'] == callback_values['rate']
assert res.storage['isClosed']
assert res.storage['isBetsForWin']
assert res.storage['closedTime'] == close_running_time
assert res.storage['closedOracleTime'] == close_oracle_time
assert res.storage['closedDynamics'] == 1_250_000  # +25%

operation = res.operations[0]
assert operation['destination'] == participants['b']
assert int(operation['amount']) == storage['expirationFee']

### Оформить этот тест в unittest
- разбить все функциональные блоки / коллы на отдельные функции (без test_, может даже с адерскором в префиксе)
- вызывать их все в тест методе
- возможно это отдельный тест и отдельно случайный тест

### assert что сделку нельзя сделать после закрытия времени

### TODO: случайный тест
- генерировать много разных участников
- генерировать случайные действия, накапливать сумму, потом сравнивать что суммы правильные (по реестрам)
- в идеале даже полностью прогонять какой-то ивент:
    - прибавлять время по k минут
    - генерировать случайные события
    - потом кто-то закрывает контракт
    - потом выводят средства, суммируются выходы с транзакций, проверяется что всё сходится
- генерировать разное время и проверять что ошибка если время не в интервале
- запускать этот случайный тест несколько раз
- лучше несколько простых случайных тестов, каждый из которых тестит какую-то часть
- хотя можно один полный с различными вариантами ставок


### TODO: make script to compile and run tests from tests folder!

желательно бы как-то вообще все безумные тесты провести:
- попытка перезапустить Measure после закрытия, попытка запустить этот процесс до завершения ставок
- любые вызовы контрактов нужно попробовать в разные периоды времени и убедиться что оно не сработает